In [13]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import pandas as pd
url = "https://finviz.com/quote.ashx?t=GOOGL"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table_data = []

contentTable = soup.find(class_ = "fullview-news-outer")
rows = contentTable.find_all('td')
for row in rows:
    data = row.get_text()
    table_data.append(data)
    
length_of_data = len(table_data)
    
headlines_dictionary = {}
for i in range(0, length_of_data, 2):
    time_date = table_data[i]
    time_date_key = time_date.replace(u'\xa0', u' ').strip()
    headline = table_data[i+1]
    
    headlines_dictionary[time_date_key] = headline
    
df = pd.DataFrame.from_dict(headlines_dictionary, orient = 'index')
df.head()

2019-10-09 22:04:49 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): finviz.com
2019-10-09 22:04:51 [urllib3.connectionpool] DEBUG: https://finviz.com:443 "GET /quote.ashx?t=GOOGL HTTP/1.1" 200 19733


,0
Oct-09-19 09:38AM,Dow Jones Today: Stocks Bounce As China Seeks ...
09:34AM,To Live or Die by Google Search Brings an Esca...
08:57AM,"Maxar, ABM, Amazon and Alphabet highlighted as..."
08:56AM,Why Once High-Flying FAANGs Dragging Down Mark...
08:37AM,"The Zacks Analyst Blog Highlights: Nvidia, AMD..."


In [ ]:
# ffill
# sentiment col.

In [4]:
# NLTK VADER for sentiment analysis
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# New words and values
new_words = {
    'crushes': 10,
    'beats': 5,
    'misses': -5,
    'trouble': -10,
    'falls': -100,
}
# Instantiate the sentiment intensity analyzer with the existing lexicon
vader = SentimentIntensityAnalyzer()
# Update the lexicon
# nltk.download('stopwords')
vader.lexicon.update(new_words)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sebh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
# Use these column names
columns = ['ticker', 'date', 'time', 'headline']
# Convert the list of lists into a DataFrame
scored_news = pd.DataFrame(parsed_news,columns=columns)
# Iterate through the headlines and get the polarity scores
scores = [vader.polarity_scores(news[3]) for news in parsed_news]
# print(scores)
# Convert the list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)
# print(scores_df)
# Join the DataFrames
scored_news = scored_news.join(scores_df)
# Convert the date column from string to datetime
scored_news['date'] = pd.to_datetime(scored_news.date).dt.date

NameError: name 'pd' is not defined

In [ ]:
# random forest classifier

In [ ]:
# google prices over 8 days -> target variable